In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 22.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [20, 6]

import warnings
warnings.filterwarnings('ignore')

In [3]:
# 전월세 데이터
data2 = pd.read_excel('data/NewRent.xlsx')
del data2['Unnamed: 0']
data2.head(2)

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,보증금(만원),월세(만원),층,건축년도,도로명,분기
0,서울특별시 강남구 개포동,SH대치1단지아파트,월세,30,2018-03-01,"3,865",17,11,1991.0,개포로109길 5,2018/1
1,서울특별시 강남구 개포동,개포2차현대아파트,월세,80,2018-02-01,"39,000",30,4,1988.0,언주로 103,2018/1


In [4]:
# id제작
data2['ID'] = data2['시군구'] + ' ' + data2['단지명'] + ' ' + data2['전용면적(㎡)'].astype('str')
data2.head(2)

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,보증금(만원),월세(만원),층,건축년도,도로명,분기,ID
0,서울특별시 강남구 개포동,SH대치1단지아파트,월세,30,2018-03-01,"3,865",17,11,1991.0,개포로109길 5,2018/1,서울특별시 강남구 개포동 SH대치1단지아파트 30
1,서울특별시 강남구 개포동,개포2차현대아파트,월세,80,2018-02-01,"39,000",30,4,1988.0,언주로 103,2018/1,서울특별시 강남구 개포동 개포2차현대아파트 80


In [5]:
# 분기설정
data2['계약년월'] = pd.to_datetime(data2['계약년월'], format='%Y%m')
data2['분기'], data2['년도'] = (data2['계약년월'].dt.quarter, data2['계약년월'].dt.year)
data2['분기'] = data2[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
data2.head(5)

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,보증금(만원),월세(만원),층,건축년도,도로명,분기,ID,년도
0,서울특별시 강남구 개포동,SH대치1단지아파트,월세,30,2018-03-01,"3,865",17,11,1991.0,개포로109길 5,2018/1,서울특별시 강남구 개포동 SH대치1단지아파트 30,2018
1,서울특별시 강남구 개포동,개포2차현대아파트,월세,80,2018-02-01,"39,000",30,4,1988.0,언주로 103,2018/1,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
2,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-04-01,"52,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
3,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"50,000",0,5,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
4,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"56,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018


In [6]:
# 월세데이터 제거
data2.drop(data2[data2['전월세구분'] == '월세'].index, inplace=True)
data2.head()

,시군구,단지명,전월세구분,전용면적(㎡),계약년월,보증금(만원),월세(만원),층,건축년도,도로명,분기,ID,년도
2,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-04-01,"52,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
3,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"50,000",0,5,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
4,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"56,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80,2018
5,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,전세,70,2018-01-01,"38,000",0,3,1987.0,언주로 3,2018/1,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 70,2018
6,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,전세,80,2018-01-01,"43,000",0,1,1987.0,언주로 3,2018/1,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,2018


In [7]:
# 자료형확인
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579046 entries, 2 to 864924
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   시군구      579046 non-null  object        
 1   단지명      578129 non-null  object        
 2   전월세구분    579046 non-null  object        
 3   전용면적(㎡)  579046 non-null  int64         
 4   계약년월     579046 non-null  datetime64[ns]
 5   보증금(만원)  579046 non-null  object        
 6   월세(만원)   579046 non-null  object        
 7   층        579046 non-null  int64         
 8   건축년도     578907 non-null  float64       
 9   도로명      579046 non-null  object        
 10  분기       579046 non-null  object        
 11  ID       578129 non-null  object        
 12  년도       579046 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(8)
memory usage: 61.8+ MB


In [8]:
# 보증금을 정수로
data2['보증금(만원)'] = data2['보증금(만원)'].str.replace(' ', '')
data2['보증금(만원)'] = data2['보증금(만원)'].str.replace(',', '')
data2['보증금(만원)'].astype(int)

2         52000
3         50000
4         56000
5         38000
6         43000
          ...  
864918    45000
864919    32500
864920    35000
864923    29400
864924    23000
Name: 보증금(만원), Length: 579046, dtype: int32

In [9]:
# 피벗테이블
# 평균값은 오류가 발생해서 중간값으로 찾음(추후 변경 고려)
table = pd.pivot_table(data2, values=['보증금(만원)'], index=['분기'], columns=['ID'], aggfunc=np.median)

In [10]:
# 확인
table

보증금(만원)                                  \
ID     서울특별시 강남구 개포동 개포2차현대아파트 80 서울특별시 강남구 개포동 개포6차우성아파트1동~8동 50   
분기                                                                  
2018/1                        NaN                             NaN   
2018/2                    52000.0                         39000.0   
2018/3                    51500.0                         40000.0   
2018/4                        NaN                         31000.0   
2019/1                    50000.0                         26000.0   
2019/2                    50000.0                         30000.0   
2019/3                    50000.0                         25000.0   
2019/4                    60000.0                             NaN   
2020/1                    53000.0                             NaN   
2020/2                    65000.0                         31500.0   
2020/3                    60000.0                             NaN   
2020/4                    70000.0                             NaN   
2021/1                    73000.0                             NaN   
2021/2                    52500.0                         34000.0   
2021/3                    55200.0                         50000.0   
2021/4                    50400.0                             NaN   
2022/1                    64000.0                             NaN   
2022/2                    61650.0                         36000.0   

                                                                        \
ID     서울특별시 강남구 개포동 개포6차우성아파트1동~8동 70 서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80   
분기                                                                       
2018/1                         39000.0                         44000.0   
2018/2                         39000.0                         45000.0   
2018/3                             NaN                         42000.0   
2018/4                         36000.0                         43000.0   
2019/1                             NaN                         37000.0   
2019/2                         30500.0                         37000.0   
2019/3                         30000.0                         42000.0   
2019/4                         36000.0                         42500.0   
2020/1                             NaN                         44500.0   
2020/2                         37500.0                         36000.0   
2020/3                         38000.0                         44000.0   
2020/4                         36500.0                         44250.0   
2021/1                         47500.0                         55000.0   
2021/2                         30000.0                         55000.0   
2021/3                         34125.0                         50000.0   
2021/4                         38000.0                         77500.0   
2022/1                         39500.0                         47250.0   
2022/2                         57500.0                         61000.0   

                                                              \
ID     서울특별시 강남구 개포동 개포더샵트리에 110 서울특별시 강남구 개포동 개포래미안포레스트 100   
분기                                                             
2018/1                       NaN                         NaN   
2018/2                       NaN                         NaN   
2018/3                       NaN                         NaN   
2018/4                  100000.0                         NaN   
2019/1                       NaN                         NaN   
2019/2                       NaN                         NaN   
2019/3                       NaN                         NaN   
2019/4                       NaN                         NaN   
2020/1                       NaN                         NaN   
2020/2                       NaN                    150000.0   
2020/3                       NaN                    170000.0   
2020/4                       NaN                    170000.0   
2021/1                       NaN                    175000.0 

In [11]:
# 전세데이터 전처리
table.fillna(method='ffill', inplace=True)
table.fillna(method='bfill', inplace=True)
data1 = table.transpose()
money_rent = data1.droplevel(axis=0, level=0)

In [12]:
# 데이터 결합
rentdic = {}
number = 0

for i1 in range(1,19):
    rentdic[f'data{i1}']=money_rent.iloc[:,[number]]
    number += 1

In [13]:
# 확인
rentdic['data1']

분기,2018/1
ID,
서울특별시 강남구 개포동 개포2차현대아파트 80,52000.0
서울특별시 강남구 개포동 개포6차우성아파트1동~8동 50,39000.0
서울특별시 강남구 개포동 개포6차우성아파트1동~8동 70,39000.0
서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,44000.0
서울특별시 강남구 개포동 개포더샵트리에 110,100000.0
...,...
서울특별시 중랑구 중화동 한영 60,20000.0
서울특별시 중랑구 중화동 한영 70,25000.0
서울특별시 중랑구 중화동 한영 80,5000.0


In [14]:
# 기존에 있던 코드
station = pd.read_excel('data/station_total.xlsx')
where = pd.read_excel('data/03_결측치처리후_주소지_4천개.xlsx')
where.drop(['Unnamed: 0'], axis=1, inplace=True)
stay_df = pd.DataFrame({'위도':station['lat'].values,'경도':station['lng'].values})
address_df = pd.DataFrame({'위도':where['위도'].values,'경도':where['경도'].values})

# 이하 데이터 전처리
base = pd.read_excel('data/ID있는버전.xlsx')
base['면적당매매가'] = base['거래금액(만원)'] / base['전용면적(㎡)']
base['저층여부'] = np.where(base['층']<5, '저층', '고층')
base['계약년월'] = pd.to_datetime(base['계약년월'], format='%Y%m')
base['분기'], base['년도'] = (base['계약년월'].dt.quarter, base['계약년월'].dt.year)
base['분기'] = base[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del base['년도']
del base['계약일']
base.drop(base[base['전용면적(㎡)']<=10].index, inplace=True)

# 거래금액, 전용면적, 건축년도 모두 사용해 보기로 함
base['계약년월'] = pd.to_datetime(base['계약년월'], format='%Y%m')
base['분기'], base['년도'] = (base['계약년월'].dt.quarter, base['계약년월'].dt.year)
base['분기'] = base[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del base['년도']
table1 = pd.pivot_table(base, values=['거래금액(만원)'], index=['분기'], columns=['ID'], aggfunc=np.mean)
table2 = pd.pivot_table(base, values=['전용면적(㎡)'], index=['분기'], columns=['ID'], aggfunc=np.mean)
table3 = pd.pivot_table(base, values=['건축년도'], index=['분기'], columns=['ID'], aggfunc=np.mean)
check_null1 = table1.isna().sum() / 18
check_null2 = table2.isna().sum() / 18
check_null3 = table3.isna().sum() / 18
check_null1[check_null1 >= 0.5]
check_null2[check_null2 >= 0.5]
check_null3[check_null3 >= 0.5]
remove_cols1 = check_null1[check_null1 >= 0.5].keys()
remove_cols2 = check_null2[check_null2 >= 0.5].keys()
remove_cols3 = check_null3[check_null3 >= 0.5].keys()
table1 = table1.drop(remove_cols1, axis=1)
table2 = table2.drop(remove_cols2, axis=1)
table3 = table3.drop(remove_cols3, axis=1)
table1.fillna(method='ffill', inplace=True)
table1.fillna(method='bfill', inplace=True)
table2.fillna(method='ffill', inplace=True)
table2.fillna(method='bfill', inplace=True)
table3.fillna(method='ffill', inplace=True)
table3.fillna(method='bfill', inplace=True)
data1 = table1.transpose()
data2 = table2.transpose()
data3 = table3.transpose()
money = data1.droplevel(axis=0, level=0)
width = data2.droplevel(axis=0, level=0)
year = data3.droplevel(axis=0, level=0)
data = pd.concat([money,width,year], axis=1)
data = data.round(1)

In [15]:
# 데이터 결합
dic = {}
number = 0

for i1 in range(1,19):
    dic[f'data{i1}']=data.iloc[:,[number, number+18, number+36]]
    number += 1

In [16]:
# 거리계산
g = []

for i in range(len(stay_df)):
    st_name = station['name'][i]
    st_line = station['line'][i]
    start = (float(stay_df['위도'][i]), float(stay_df['경도'][i]))
    for j in range(len(address_df)):
        apt_name = where['ID'][j]
        goal = (float(address_df['위도'][j]), float(address_df['경도'][j]))
        geoly = haversine(start, goal)
        r_start = np.round(start, 2)
        r_goal = np.round(goal, 2)
        r_geoly = np.round(geoly, 2)
        if r_geoly <=0.5:
            g.append([st_name, st_line, r_start, apt_name, r_goal, r_geoly])
        else:
            pass

In [17]:
# 데이터프레임화
g_df = pd.DataFrame(g, columns=['역 이름', '역 호선', '역 좌표', '아파트 이름', '아파트 좌표', '역까지의 거리'])

def search(x):
    x = g_df[g_df['아파트 이름']==x]
    return x

# 인접역 산출
p1 = []
n = 0
for i in range(len(base['ID'])):
    a = base['ID'].iloc[n]
    b = len(search(base['ID'].iloc[n]))
    p1.append([a,b])
    n+=1
# test = pd.concat([dic['data1'], p1], axis=1)
p1_df = pd.DataFrame(p1, columns=['ID','인접역 수'])

In [18]:
# 각 분기별 별도의 데이터프레임화, 컬럼명 붙이기, 전세가와 인접역 붙이기, 중복값 제거
tmp1 = pd.DataFrame(dic['data1'])
ttmp1 = pd.DataFrame(rentdic['data1'])
tmp1.reset_index(inplace=True)
ttmp1.reset_index(inplace=True)
tmp1.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp1.columns = ['ID', '전세금(만원)']
dff1 = [tmp1, ttmp1, p1_df]
main1 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff1)
main1.drop_duplicates(inplace=True)

tmp2 = pd.DataFrame(dic['data2'])
ttmp2 = pd.DataFrame(rentdic['data2'])
tmp2.reset_index(inplace=True)
ttmp2.reset_index(inplace=True)
tmp2.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp2.columns = ['ID', '전세금(만원)']
dff2 = [tmp2, ttmp2, p1_df]
main2 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff2)
main2.drop_duplicates(inplace=True)

tmp3 = pd.DataFrame(dic['data3'])
ttmp3 = pd.DataFrame(rentdic['data3'])
tmp3.reset_index(inplace=True)
ttmp3.reset_index(inplace=True)
tmp3.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp3.columns = ['ID', '전세금(만원)']
dff3 = [tmp3, ttmp3, p1_df]
main3 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff3)
main3.drop_duplicates(inplace=True)

tmp4 = pd.DataFrame(dic['data4'])
ttmp4 = pd.DataFrame(rentdic['data4'])
tmp4.reset_index(inplace=True)
ttmp4.reset_index(inplace=True)
tmp4.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp4.columns = ['ID', '전세금(만원)']
dff4 = [tmp4, ttmp4, p1_df]
main4 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff4)
main4.drop_duplicates(inplace=True)

tmp5 = pd.DataFrame(dic['data5'])
ttmp5 = pd.DataFrame(rentdic['data5'])
tmp5.reset_index(inplace=True)
ttmp5.reset_index(inplace=True)
tmp5.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp5.columns = ['ID', '전세금(만원)']
dff5 = [tmp5, ttmp5, p1_df]
main5 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff5)
main5.drop_duplicates(inplace=True)

tmp6 = pd.DataFrame(dic['data6'])
ttmp6 = pd.DataFrame(rentdic['data6'])
tmp6.reset_index(inplace=True)
ttmp6.reset_index(inplace=True)
tmp6.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp6.columns = ['ID', '전세금(만원)']
dff6 = [tmp6, ttmp6, p1_df]
main6 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff6)
main6.drop_duplicates(inplace=True)

tmp7 = pd.DataFrame(dic['data7'])
ttmp7 = pd.DataFrame(rentdic['data7'])
tmp7.reset_index(inplace=True)
ttmp7.reset_index(inplace=True)
tmp7.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp7.columns = ['ID', '전세금(만원)']
dff7 = [tmp7, ttmp7, p1_df]
main7 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff7)
main7.drop_duplicates(inplace=True)

tmp8 = pd.DataFrame(dic['data8'])
ttmp8 = pd.DataFrame(rentdic['data8'])
tmp8.reset_index(inplace=True)
ttmp8.reset_index(inplace=True)
tmp8.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp8.columns = ['ID', '전세금(만원)']
dff8 = [tmp8, ttmp8, p1_df]
main8 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff8)
main8.drop_duplicates(inplace=True)

tmp9 = pd.DataFrame(dic['data9'])
ttmp9 = pd.DataFrame(rentdic['data9'])
tmp9.reset_index(inplace=True)
ttmp9.reset_index(inplace=True)
tmp9.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp9.columns = ['ID', '전세금(만원)']
dff9 = [tmp9, ttmp9, p1_df]
main9 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff9)
main9.drop_duplicates(inplace=True)

tmp10 = pd.DataFrame(dic['data10'])
ttmp10 = pd.DataFrame(rentdic['data10'])
tmp10.reset_index(inplace=True)
ttmp10.reset_index(inplace=True)
tmp10.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp10.columns = ['ID', '전세금(만원)']
dff10 = [tmp10, ttmp10, p1_df]
main10 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff10)
main10.drop_duplicates(inplace=True)

tmp11 = pd.DataFrame(dic['data11'])
ttmp11 = pd.DataFrame(rentdic['data11'])
tmp11.reset_index(inplace=True)
ttmp11.reset_index(inplace=True)
tmp11.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp11.columns = ['ID', '전세금(만원)']
dff11 = [tmp11, ttmp11, p1_df]
main11 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff11)
main11.drop_duplicates(inplace=True)

tmp12 = pd.DataFrame(dic['data12'])
ttmp12 = pd.DataFrame(rentdic['data12'])
tmp12.reset_index(inplace=True)
ttmp12.reset_index(inplace=True)
tmp12.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp12.columns = ['ID', '전세금(만원)']
dff12 = [tmp12, ttmp12, p1_df]
main12 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff12)
main12.drop_duplicates(inplace=True)

tmp13 = pd.DataFrame(dic['data13'])
ttmp13 = pd.DataFrame(rentdic['data13'])
tmp13.reset_index(inplace=True)
ttmp13.reset_index(inplace=True)
tmp13.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp13.columns = ['ID', '전세금(만원)']
dff13 = [tmp13, ttmp13, p1_df]
main13 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff13)
main13.drop_duplicates(inplace=True)

tmp14 = pd.DataFrame(dic['data14'])
ttmp14 = pd.DataFrame(rentdic['data14'])
tmp14.reset_index(inplace=True)
ttmp14.reset_index(inplace=True)
tmp14.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp14.columns = ['ID', '전세금(만원)']
dff14 = [tmp14, ttmp14, p1_df]
main14 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff14)
main14.drop_duplicates(inplace=True)

tmp15 = pd.DataFrame(dic['data15'])
ttmp15 = pd.DataFrame(rentdic['data15'])
tmp15.reset_index(inplace=True)
ttmp15.reset_index(inplace=True)
tmp15.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp15.columns = ['ID', '전세금(만원)']
dff15 = [tmp15, ttmp15, p1_df]
main15 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff15)
main15.drop_duplicates(inplace=True)

tmp16 = pd.DataFrame(dic['data16'])
ttmp16 = pd.DataFrame(rentdic['data16'])
tmp16.reset_index(inplace=True)
ttmp16.reset_index(inplace=True)
tmp16.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp16.columns = ['ID', '전세금(만원)']
dff16 = [tmp16, ttmp16, p1_df]
main16 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff16)
main16.drop_duplicates(inplace=True)

tmp17 = pd.DataFrame(dic['data17'])
ttmp17 = pd.DataFrame(rentdic['data17'])
tmp17.reset_index(inplace=True)
ttmp17.reset_index(inplace=True)
tmp17.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp17.columns = ['ID', '전세금(만원)']
dff17 = [tmp17, ttmp17, p1_df]
main17 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff17)
main17.drop_duplicates(inplace=True)

tmp18 = pd.DataFrame(dic['data18'])
ttmp18 = pd.DataFrame(rentdic['data18'])
tmp18.reset_index(inplace=True)
ttmp18.reset_index(inplace=True)
tmp18.columns = ['ID', '거래금액(만원)', '전용면적', '건축년도']
ttmp18.columns = ['ID', '전세금(만원)']
dff18 = [tmp18, ttmp18, p1_df]
main18 = reduce(lambda left, right: pd.merge(left, right, on='ID'), dff18)
main18.drop_duplicates(inplace=True)

In [19]:
# 확인1
main1

,ID,거래금액(만원),전용면적,건축년도,전세금(만원),인접역 수
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,80.0,1987.0,44000.0,0
24,서울특별시 강남구 개포동 개포주공1단지 40,145736.8,40.0,1982.0,9500.0,1
146,서울특별시 강남구 개포동 개포주공5단지 60,136666.7,60.0,1983.0,43000.0,1
171,서울특별시 강남구 개포동 개포주공5단지 80,177416.7,80.0,1983.0,45000.0,1
199,서울특별시 강남구 개포동 개포주공6단지 50,125900.0,50.0,1983.0,37000.0,1
...,...,...,...,...,...,...
214262,서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,60.0,1998.0,2500.0,1
214281,서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,80.0,2005.0,15000.0,1
214302,서울특별시 중랑구 중화동 한신아파트 50,31756.2,50.0,1997.0,25000.0,5
214385,서울특별시 중랑구 중화동 한신아파트 60,36534.6,60.0,1997.1,28000.0,5


In [20]:
# 확인2
main18

,ID,거래금액(만원),전용면적,건축년도,전세금(만원),인접역 수
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,218666.7,80.0,1987.0,61000.0,0
24,서울특별시 강남구 개포동 개포주공1단지 40,180000.0,40.0,1982.0,9500.0,1
146,서울특별시 강남구 개포동 개포주공5단지 60,202800.0,60.0,1983.0,45000.0,1
171,서울특별시 강남구 개포동 개포주공5단지 80,282250.0,80.0,1983.0,61000.0,1
199,서울특별시 강남구 개포동 개포주공6단지 50,189000.0,50.0,1983.0,42500.0,1
...,...,...,...,...,...,...
214262,서울특별시 중랑구 중화동 중화동건영아파트 60,47500.0,60.0,1998.0,24150.0,1
214281,서울특별시 중랑구 중화동 태능에셈빌 80,71000.0,80.0,2005.0,15000.0,1
214302,서울특별시 중랑구 중화동 한신아파트 50,70000.0,50.0,1997.0,37000.0,5
214385,서울특별시 중랑구 중화동 한신아파트 60,73000.0,60.0,1997.0,33600.0,5


In [21]:
# ID제거, ID가 문자열이라서 스케일링에 문제가 생김
unmain1 = main1.drop(columns='ID')
unmain2 = main2.drop(columns='ID')
unmain3 = main3.drop(columns='ID')
unmain4 = main4.drop(columns='ID')
unmain5 = main5.drop(columns='ID')
unmain6 = main6.drop(columns='ID')
unmain7 = main7.drop(columns='ID')
unmain8 = main8.drop(columns='ID')
unmain9 = main9.drop(columns='ID')
unmain10 = main10.drop(columns='ID')
unmain11 = main11.drop(columns='ID')
unmain12 = main12.drop(columns='ID')
unmain13 = main13.drop(columns='ID')
unmain14 = main14.drop(columns='ID')
unmain15 = main15.drop(columns='ID')
unmain16 = main16.drop(columns='ID')
unmain17 = main17.drop(columns='ID')
unmain18 = main18.drop(columns='ID')

# 스케일링과 클러스터링
# 여전히 군집고정은 해결 못함
mm = MinMaxScaler()
mm_scaled1 = mm.fit_transform(unmain1)
mm_scaled2 = mm.fit_transform(unmain2)
mm_scaled3 = mm.fit_transform(unmain3)
mm_scaled4 = mm.fit_transform(unmain4)
mm_scaled5 = mm.fit_transform(unmain5)
mm_scaled6 = mm.fit_transform(unmain6)
mm_scaled7 = mm.fit_transform(unmain7)
mm_scaled8 = mm.fit_transform(unmain8)
mm_scaled9 = mm.fit_transform(unmain9)
mm_scaled10 = mm.fit_transform(unmain10)
mm_scaled11 = mm.fit_transform(unmain11)
mm_scaled12 = mm.fit_transform(unmain12)
mm_scaled13 = mm.fit_transform(unmain13)
mm_scaled14 = mm.fit_transform(unmain14)
mm_scaled15 = mm.fit_transform(unmain15)
mm_scaled16 = mm.fit_transform(unmain16)
mm_scaled17 = mm.fit_transform(unmain17)
mm_scaled18 = mm.fit_transform(unmain18)

kmeans = KMeans(n_clusters=5, init='k-means++', random_state=np.random.RandomState(seed=1))

clusters1 = kmeans.fit(mm_scaled1)
unmain1['cluster'] = clusters1.labels_
clusters2 = kmeans.fit(mm_scaled2)
unmain2['cluster'] = clusters2.labels_
clusters3 = kmeans.fit(mm_scaled3)
unmain3['cluster'] = clusters3.labels_
clusters4 = kmeans.fit(mm_scaled4)
unmain4['cluster'] = clusters4.labels_
clusters5 = kmeans.fit(mm_scaled5)
unmain5['cluster'] = clusters5.labels_
clusters6 = kmeans.fit(mm_scaled6)
unmain6['cluster'] = clusters6.labels_
clusters7 = kmeans.fit(mm_scaled7)
unmain7['cluster'] = clusters7.labels_
clusters8 = kmeans.fit(mm_scaled8)
unmain8['cluster'] = clusters8.labels_
clusters9 = kmeans.fit(mm_scaled9)
unmain9['cluster'] = clusters9.labels_
clusters10 = kmeans.fit(mm_scaled10)
unmain10['cluster'] = clusters10.labels_
clusters11 = kmeans.fit(mm_scaled11)
unmain11['cluster'] = clusters11.labels_
clusters12 = kmeans.fit(mm_scaled12)
unmain12['cluster'] = clusters12.labels_
clusters13 = kmeans.fit(mm_scaled13)
unmain13['cluster'] = clusters13.labels_
clusters14 = kmeans.fit(mm_scaled14)
unmain14['cluster'] = clusters14.labels_
clusters15 = kmeans.fit(mm_scaled15)
unmain15['cluster'] = clusters15.labels_
clusters16 = kmeans.fit(mm_scaled16)
unmain16['cluster'] = clusters16.labels_
clusters17 = kmeans.fit(mm_scaled17)
unmain17['cluster'] = clusters17.labels_
clusters18 = kmeans.fit(mm_scaled18)
unmain18['cluster'] = clusters18.labels_

In [22]:
# 확인
unmain1

,거래금액(만원),전용면적,건축년도,전세금(만원),인접역 수,cluster
0,129125.0,80.0,1987.0,44000.0,0,0
24,145736.8,40.0,1982.0,9500.0,1,0
146,136666.7,60.0,1983.0,43000.0,1,0
171,177416.7,80.0,1983.0,45000.0,1,0
199,125900.0,50.0,1983.0,37000.0,1,0
...,...,...,...,...,...,...
214262,25000.0,60.0,1998.0,2500.0,1,1
214281,35000.0,80.0,2005.0,15000.0,1,1
214302,31756.2,50.0,1997.0,25000.0,5,2
214385,36534.6,60.0,1997.1,28000.0,5,2


In [23]:
# ID 다시 붙여주기
unmain1.set_index(main1['ID'], inplace=True)
unmain2.set_index(main1['ID'], inplace=True)
unmain3.set_index(main1['ID'], inplace=True)
unmain4.set_index(main1['ID'], inplace=True)
unmain5.set_index(main1['ID'], inplace=True)
unmain6.set_index(main1['ID'], inplace=True)
unmain7.set_index(main1['ID'], inplace=True)
unmain8.set_index(main1['ID'], inplace=True)
unmain9.set_index(main1['ID'], inplace=True)
unmain10.set_index(main1['ID'], inplace=True)
unmain11.set_index(main1['ID'], inplace=True)
unmain12.set_index(main1['ID'], inplace=True)
unmain13.set_index(main1['ID'], inplace=True)
unmain14.set_index(main1['ID'], inplace=True)
unmain15.set_index(main1['ID'], inplace=True)
unmain16.set_index(main1['ID'], inplace=True)
unmain17.set_index(main1['ID'], inplace=True)
unmain18.set_index(main1['ID'], inplace=True)

In [24]:
# 확인
unmain1

,거래금액(만원),전용면적,건축년도,전세금(만원),인접역 수,cluster
ID,,,,,,
서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,129125.0,80.0,1987.0,44000.0,0,0
서울특별시 강남구 개포동 개포주공1단지 40,145736.8,40.0,1982.0,9500.0,1,0
서울특별시 강남구 개포동 개포주공5단지 60,136666.7,60.0,1983.0,43000.0,1,0
서울특별시 강남구 개포동 개포주공5단지 80,177416.7,80.0,1983.0,45000.0,1,0
서울특별시 강남구 개포동 개포주공6단지 50,125900.0,50.0,1983.0,37000.0,1,0
...,...,...,...,...,...,...
서울특별시 중랑구 중화동 중화동건영아파트 60,25000.0,60.0,1998.0,2500.0,1,1
서울특별시 중랑구 중화동 태능에셈빌 80,35000.0,80.0,2005.0,15000.0,1,1
서울특별시 중랑구 중화동 한신아파트 50,31756.2,50.0,1997.0,25000.0,5,2


In [25]:
# 하나로 합치기
address = pd.merge(unmain1, where, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain2, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain3, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain4, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain5, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain6, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain7, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain8, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain9, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain10, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain11, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain12, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain13, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain14, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain15, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain16, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain17, address, left_on='ID', right_on='ID', how='inner')
address = pd.merge(unmain18, address, left_on='ID', right_on='ID', how='inner')

In [26]:
# 엑셀로 내보내기
address.to_excel('data/NEW_Mega_address.xlsx')

이전 코드와 차이점  
1. 전세가가 추가됨

이번 코드도 엑셀로 내보낸 후 엑셀에서 처리한 부분 다수 있음